# Use Graphviz

In [ ]:
import sys

import graphviz
from IPython.display import display

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")

import yaml

from PhotonicsAI.Photon import utils
from PhotonicsAI.Photon.DemoPDK import *

dot_string = """
graph new_circuit {
  rankdir=LR;
  node [shape=record];
  C1 [label="{{<o1> o1} | C1: _mmi1x2 | {<o2> o2|<o3> o3}}"];
  C2 [label="{{<o1> o1} | C2: mzi_1x1_pindiode_cband | {<o2> o2}}"];
  C3 [label="{{<o1> o1} | C3: mzi_1x1_pindiode_cband | {<o2> o2}}"];
  C1:o3 -- C3:o1;
  C1:o2 -- C2:o1;
}
"""

dot_string_sized = """
graph new_circuit {
  rankdir=LR;
  C1 [width=2.0, height=.2, shape=record, fixedsize=true];
  C2 [width=61.5, height=8.8, shape=record, fixedsize=true];
  C3 [width=61.5, height=8.8, shape=record, fixedsize=true];

  C1 [label="{{<o1> o1} | C1: _mmi1x2 | {<o2> o2|<o3> o3}}"];
  C2 [label="{{<o1> o1} | C2: mzi_1x1_pindiode_cband | {<o2> o2}}"];
  C3 [label="{{<o1> o1} | C3: mzi_1x1_pindiode_cband | {<o2> o2}}"];

  C1:o3 -- C3:o1;
  C1:o2 -- C2:o1;
}
"""


def add_node_sizes(dot_string, node_dimensions):
    lines = dot_string.split("\n")
    output_lines = []
    nodes_added = set()

    for line in lines:
        stripped_line = line.strip()
        if (
            stripped_line.startswith("C")
            and "[" in stripped_line
            and stripped_line.endswith("];")
        ):
            node_name = stripped_line.split("[")[0].strip()
            if node_name in node_dimensions and node_name not in nodes_added:
                width, height = node_dimensions[node_name]
                size_line = f"  {node_name} [width={width}, height={height}, shape=record, fixedsize=true];"
                output_lines.append(size_line)
                nodes_added.add(node_name)
        output_lines.append(line)

    return "\n".join(output_lines)


def multiply_node_dimensions(node_dimensions, factor=0.01):
    return {
        node: (round(width * factor, 3), round(height * factor, 3))
        for node, (width, height) in node_dimensions.items()
    }


sizes_dict = {"C1": (20.0, 2.5), "C2": (615.6, 88.05), "C3": (615.6, 88.05)}
dot_string_sized = add_node_sizes(
    dot_string, multiply_node_dimensions(sizes_dict, 0.01)
)
dot = graphviz.Source(dot_string_sized)
display(dot)

graphviz_node_coordinates = utils.get_graphviz_placements(dot_string_sized)
graphviz_node_coordinates = multiply_node_dimensions(
    graphviz_node_coordinates, 100 / 72
)

print(graphviz_node_coordinates)

In [ ]:
netlist = """
instances:
  C1:
    component: _mmi1x2
    info:
      ports: 1x2
  C2:
    component: mzi_1x1_pindiode_cband
    info:
      ports: 1x1
  C3:
    component: mzi_1x1_pindiode_cband
    info:
      ports: 1x1
name: new_circuit
routes:
  optical:
    links:
      C1,o3: C3,o1
      C1,o2: C2,o1
"""


def add_placements_to_yaml(yaml_str, placements):
    data = yaml.safe_load(yaml_str)

    # Add placements to the data
    data["placements"] = {
        key: {"x": value[0], "y": value[1]} for key, value in placements.items()
    }

    return yaml.dump(data, default_flow_style=False)


# graphviz_node_coordinates = {'C1': (105.208, 63.194), 'C2': (426.556, 102.083), 'C3': (426.556, 25.694)}

netlist_new = add_placements_to_yaml(netlist, graphviz_node_coordinates)
print(netlist_new)

# print(graphviz_node_coordinates.items())
# gds_fig, html_gds_fig, gds_netlist = yaml_netlist_to_gds(netlist, ignore_links=False)
# display(gds_fig)

In [ ]:
import sys

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")
from PhotonicsAI.Photon import utils
from PhotonicsAI.Photon.DemoPDK import *

# c = DemoPDK.get_component('wdm_mzi1x4')
# print(c.dxsize, c.dysize)
# c.write_gds("demo.gds")

netlist = """
instances:
  C1:
    component: wdm_mzi1x4
    info:
      dx: 1190.4
      dy: 362.67
      ports: 1x4
  C2:
    component: dc_mzi_2x2_heater_doped_si_cband
    info:
      dx: 274.0
      dy: 98.4
      ports: 2x2
  C3:
    component: dc_mzi_2x2_heater_doped_si_cband
    info:
      dx: 274.0
      dy: 98.4
      ports: 2x2
  C4:
    component: dc_mzi_2x2_heater_doped_si_cband
    info:
      dx: 274.0
      dy: 98.4
      ports: 2x2
  C5:
    component: dc_mzi_2x2_heater_doped_si_cband
    info:
      dx: 274.0
      dy: 98.4
      ports: 2x2
placements:
  C1:
    x: 714.236
    y: 304.181
  C2:
    x: 1642.917
    y: 493.069
  C3:
    x: 1642.917
    y: 348.625
  C4:
    x: 1642.917
    y: 204.181
  C5:
    x: 1642.917
    y: 59.735
routes:
  optical:
    links:
      C1,o2: C2,o2
      C1,o3: C3,o2
      C1,o4: C4,o2
      C1,o5: C5,o2
"""

c = gf.read.from_yaml(netlist)
c.plot()